In [6]:
import kagglehub
import pandas as pd

# Download latest version
path_results = kagglehub.dataset_download("martj42/international-football-results-from-1872-to-2017")

results_df = pd.read_csv(path_results + "/results.csv")

results_df["date"] = pd.to_datetime(results_df["date"])

path_countries = kagglehub.dataset_download("phjulien/national-football-team-affiliations")


In [7]:
afc = [
    team.strip()
    for team in open(f'{path_countries}/AFC', 'r').read().split('\n')
]
caf = [
    team.strip()
    for team in  open(f'{path_countries}/CAF', 'r').read().strip().split('\n')
]
concacaf = [
    team.strip()
    for team in  open(f'{path_countries}/Concacaf', 'r').read().strip().split('\n')
]
conmebol = [
    team.strip()
    for team in  open(f'{path_countries}/Conmebol', 'r').read().strip().split('\n')
]
ofc = [
    team.strip() for team in  open(f'{path_countries}/OFC', 'r').read().strip().split('\n')
]
uefa = [
    team.strip() for team in  open(f'{path_countries}/UEFA', 'r').read().strip().split('\n')
]

In [8]:
# Fix some typos in the names and change others to match the names in the already available data
afc[afc.index('Brunei Darussalam')] = 'Brunei'
afc[afc.index('Chinese Taipei')] = 'Taiwan'
caf[caf.index('Libya1')] = "Libya"
caf[caf.index('Mauritania1')] = 'Mauritania'
caf[caf.index('Morocco1')] = 'Morocco'
caf[caf.index('Congo DR')] = 'Congo'
caf[caf.index('Swaziland')] = 'Eswatini'
concacaf[concacaf.index('US Virgin Islands')] = 'United States Virgin Islands'
uefa[uefa.index('FYR Macedonia')] = 'North Macedonia'

In [9]:
affiliations_df = pd.DataFrame(
    {
        'federation': ['AFC'] * len(afc) + ['CAF'] * len(caf) + ['CONCACAF'] * len(concacaf) +
        ['CONMEBOL'] * len(conmebol) + ['OFC'] * len(ofc) + ['UEFA'] * len(uefa),
        'region': ['Asia'] * len(afc) + ['Africa'] * len(caf) + ['North and Central America'] * len(concacaf) +
        ['South America'] * len(conmebol) + ['Oceania'] * len(ofc) + ['Europe'] * len(uefa),
        'team': afc + caf + concacaf + conmebol + ofc + uefa,
    },
)
results_df['year'] = results_df['date'].dt.year
all_teams = list(results_df['home_team'].unique())
all_teams.extend(list(results_df['away_team'].unique()))
all_teams_df = pd.DataFrame({'team': list(set(all_teams))})
all_teams_df['first_game_year'] = all_teams_df['team'].apply(
    lambda x: results_df[(results_df['home_team'] == x) | (results_df['away_team'] == x)].head(1)['year'].values[0],
)
all_teams_df = all_teams_df.merge(affiliations_df, how='left', on=['team'])
all_teams_df.sort_values(by='first_game_year', inplace=True)
all_teams_df.head()

,team,first_game_year,federation,region
217,Scotland,1872,UEFA,Europe
163,England,1872,UEFA,Europe
79,Wales,1876,UEFA,Europe
80,Northern Ireland,1882,UEFA,Europe
189,Canada,1885,CONCACAF,North and Central America
